[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/edit2d/edit2d_all_features.ipynb)

# Edit2D — All Features

Comprehensive demo of the Edit2D crop, pad, and mask tool.

In [ ]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

In [ ]:
import numpy as np
from quantem.widget import Edit2D

def make_crystal(size=256, seed=0):
    """Simulate a crystal lattice image with point defects."""
    rng = np.random.default_rng(seed)
    y, x = np.mgrid[:size, :size]
    # Two-beam lattice fringes
    img = np.cos(2 * np.pi * 0.08 * x) + np.cos(2 * np.pi * 0.08 * y)
    # Add a few bright point defects
    for _ in range(5):
        cy, cx = rng.integers(20, size - 20, size=2)
        r2 = (x - cx)**2 + (y - cy)**2
        img += 3.0 * np.exp(-r2 / 8)
    img += rng.normal(0, 0.2, (size, size))
    return img.astype(np.float32)

image = make_crystal(256)

## 1. Basic Crop

Drag the crop rectangle interactively, or set bounds programmatically.

In [ ]:
w = Edit2D(image, title="Basic Crop")
w

In [ ]:
# Read back the crop
print(f"Bounds: {w.crop_bounds}")
print(f"Size:   {w.crop_size}")
print(f"Result: {w.result.shape}")

## 2. Programmatic Bounds

Set crop bounds as `(top, left, bottom, right)` in image coordinates.

In [ ]:
Edit2D(image, bounds=(50, 50, 200, 200), title="Center Crop (150x150)")

## 3. Padding (Bounds Beyond Image)

Negative bounds or bounds exceeding image dimensions produce padding with `fill_value`.

In [ ]:
w_pad = Edit2D(image, bounds=(-30, -30, 286, 286), fill_value=0.0, title="Padded (30px border)")
w_pad

In [ ]:
result = w_pad.result
print(f"Padded result: {result.shape}")  # 316x316
print(f"Corner value (should be fill): {result[0, 0]}")
print(f"Center value (should be data): {result[30, 30]:.4f}")

## 4. Mask Mode

Paint a binary mask over the image. Masked pixels are set to `fill_value` in the result.

In [ ]:
w_mask = Edit2D(image, mode="mask", fill_value=0.0, title="Mask Mode")
w_mask

In [ ]:
# Access the mask and masked result
print(f"Mask shape: {w_mask.mask.shape}")
print(f"Masked pixels: {w_mask.mask.sum()}")
print(f"Result shape: {w_mask.result.shape}")

## 5. Multi-Image Mode

Apply the same crop/mask to multiple images at once.

In [ ]:
images = [make_crystal(256, seed=i) for i in range(3)]
w_multi = Edit2D(images, labels=["Crystal A", "Crystal B", "Crystal C"],
                 bounds=(30, 30, 220, 220), title="Multi-Image Crop")
w_multi

In [ ]:
results = w_multi.result
print(f"Number of results: {len(results)}")
for i, r in enumerate(results):
    print(f"  Image {i}: {r.shape}")

## 6. Display Options

Log scale, colormap, and scale bar.

In [ ]:
Edit2D(image, cmap="viridis", log_scale=True,
       pixel_size_angstrom=2.5, title="Viridis + Log Scale")

## 7. Replace Data with `set_image()`

Swap the underlying data while preserving display settings.

In [ ]:
w_swap = Edit2D(image, cmap="inferno", title="Original")
print(f"Before: {w_swap.height}x{w_swap.width}")

new_image = make_crystal(128, seed=42)
w_swap.set_image(new_image)
print(f"After:  {w_swap.height}x{w_swap.width}")
print(f"Cmap preserved: {w_swap.cmap}")
w_swap

## 8. State Persistence

Save and restore widget settings across sessions.

In [ ]:
w_state = Edit2D(image, cmap="plasma", bounds=(20, 30, 200, 220),
                 fill_value=5.0, title="State Demo")
w_state.summary()

In [ ]:
# Save and inspect
w_state.save("/tmp/edit2d_state.json")
print(w_state.state_dict())

In [ ]:
# Restore from file
w2 = Edit2D(image, state="/tmp/edit2d_state.json")
w2.summary()

In [ ]:
import os
os.remove("/tmp/edit2d_state.json")